In [1]:
import xlrd
import requests
import string
from xlutils.copy import copy
from bs4 import BeautifulSoup
import csv
import sys
from threading import Thread

In [2]:
def extract(i, career_href, letter):

    wb = open('cv-'+str(letter)+'.csv', mode='a', encoding='utf-8', newline="\n")
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}

    # Request Personal Profile
    start_html = requests.get(career_href, headers=headers)
    # HTML code
    Soup = BeautifulSoup(start_html.text, 'lxml')
    # all_tr: All information for one person, long <tr>
    all_tr = Soup.find_all('tr') 
    # number of all_tr
    num_tr = len(all_tr)
    # name is the last tr content
    cur_name = all_tr[num_tr-1].get_text() # name without url
    cur_name = cur_name.replace('\n','')
    index = 0
    try:
        for td in all_tr[0:num_tr-5]:
            td_text = td.get_text()
    
            all_td = td.find_all('td') # all <td>s in <tr>
            year = all_td[0].get_text().split(',')[0] # Year
            year = year.replace('—', '-')
            #print(all_td)
            title = all_td[1].get_text().split(',')[0] # Title
            # print(title)
            for a in all_td[0:-1]:
                    
                #print("First: " + first_td + "\n")
                more_a = a.find_all('a')
                if more_a:
                    info = title
                    print(more_a)
                    for content in more_a:
                        info = info + ',' + content.get_text()
                    info = info.replace('\n','')
                    

                    result = cur_name + ',' + year + ',' + info + '\n'

                    wb.write(result)
                    # wb.flush()
                    index += 1
    except:
        pass

    wb.close()

In [3]:
def replace_string(s):
    s1 = s.replace(" ", "%20")
    #s1 = s.replace("|", "%7C")
    #s1 = s.repalce("'", "%27")
    return s1

def b(letter):

    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}
    str_start = str("http://www.chinavitae.com/biography_browse.php?l=")#+str(letter)

    #range 0-26 represent a-z
    for i in (range(0, 1)):    
        # Search Url: http://www.chinavitae.com/biography_browse.php?l={A}
        alpha_url = str_start + letter# + string.ascii_uppercase[i]
        print(alpha_url)
        # Get HTML requests
        start_html = requests.get(alpha_url, headers=headers)
        # Get HTML code from requests. Soup contains all HTML code
        Soup = BeautifulSoup(start_html.text, 'lxml')
        # <td> tags contain name list
        all_td = Soup.find_all('td')
        # Column 1,2,3,4 from name search page
        for page in range(2, 6): # Loop from Col_1 to Col_4
            # All names in one search page
            cur_td = all_td[page]
            # all_a: all names in a single row
            all_a = cur_td.find_all('a')
            # For each name in a row (4 in total)
            for a in all_a: # Loop from Row_1 to Row_n
                # Title: Zakir Shohrat. A name without "_"
                title = a.get_text()
                # A post address: /biography/Zakir_Shohrat
                href = a['href']
                # Fix invalid url: " "==%20 and |==%7C and '==%27
                new_href = replace_string(href)
                # Return full url for someone
                career_href = "http://www.chinavitae.com" + new_href + "/career"
                # extract the table content into excel. i: sheet number
                extract(i, career_href, letter)

In [4]:
def fetcher(num, begin, end):

    s = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i in range(begin, end):
        b(s[i])
        print("Fetcher: " + str(num) + ", letter: " + s[i] + ". Complete.")
    print("Fetcher: " + str(num) + " totally completed.")

In [5]:
t1 = Thread(target=fetcher, args=('1', 0,6)) # ABCDEF
# t2 = Thread(target=fetcher, args=('2', 6,12)) # GHIJKL
# t3 = Thread(target=fetcher, args=('3', 12,18)) # MNOPQR
# t4 = Thread(target=fetcher, args=('4', 18,23)) # STUVWXYZ 
# t5 = Thread(target=fetcher, args=('5', 23,26)) # MNO

t1.start()
# t2.start()
# t3.start()
# t4.start()
# t5.start()

print("Main complete")

Main complete
http://www.chinavitae.com/biography_browse.php?l=A


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-4-ea82c0a2e6c4>", line 5, in fetcher
    b(s[i])
  File "<ipython-input-3-cb4fd6882c7b>", line 40, in b
    extract(i, career_href, letter)
  File "<ipython-input-2-5f0fd70f467c>", line 3, in extract
    wb = open('cv-'+str(letter)+'.csv', mode='a', encoding='utf-8', newline="\n")
TypeError: file() takes at most 3 arguments (4 given)

